<a href="https://colab.research.google.com/github/Loryana/OCProjet7/blob/main/Projet_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet
!pip install dagshub --quiet
import mlflow

In [ ]:
!mlflow --version

mlflow, version 2.17.0


In [ ]:
import dagshub
dagshub.init(repo_owner='loryana9', repo_name='Collab_Dags', mlflow=True)

Initialized MLflow to track repo "loryana9/Collab_Dags"

Repository loryana9/Collab_Dags initialized!

In [ ]:
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# Set the tracking URI.
mlflow.set_tracking_uri("https://dagshub.com/loryana9/Collab_Dags.mlflow")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2024/10/22 11:28:44 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.
Successfully registered model 'tracking-quickstart'.
2024/10/22 11:28:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


2024/10/22 11:28:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run selective-shrew-614 at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/0/runs/fe749ed4f28e43e7b1fb5d95334605e5.
2024/10/22 11:28:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/0.


In [ ]:
# Load the model back for predictions as a generic Python Function model
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [ ]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [ ]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [ ]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [ ]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [ ]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 24s 541ms/step - loss: 34.2770 - root_mean_squared_error: 5.8547
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.4475 - root_mean_squared_error: 4.9098   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 20.9788 - root_mean_squared_error: 4.5179 - val_loss: 2.6580 - val_root_mean_squared_error: 1.6303

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 2.7942 - root_mean_squared_error: 1.6716
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.5123 - root_mean_squared_error: 1.5836 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4387 - root_mean_squared_error: 1.5599 - val_loss: 1.9309 - val_root_mean_squared_error: 1.3896

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 174ms/step - loss: 1.3963 - root_mean_squared_error: 1.1817
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7013 - root_mean_squared_error: 1.3040  
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7018 - root_mean_squared_error: 1.3043 - val_loss: 1.6607 - val_root_mean_squared_e

2024/10/22 12:05:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-dove-221 at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1/runs/5a6a3efc3d7c4d07a6dc280efa1ad9e8.

2024/10/22 12:05:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1.



Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 24s 542ms/step - loss: 29.3717 - root_mean_squared_error: 5.4196
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.3596 - root_mean_squared_error: 2.5717    
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.5111 - root_mean_squared_error: 2.1956 - val_loss: 0.6811 - val_root_mean_squared_error: 0.8253

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.6561 - root_mean_squared_error: 0.8100
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6271 - root_mean_squared_error: 0.7918 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6219 - root_mean_squared_error: 0.7885 - val_loss: 0.5574 - val_root_mean_squared_error: 0.7466

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - loss: 0.5477 - root_mean_squared_error: 0.7400
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5783 - root_mean_squared_error: 0.7603  
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5744 - root_mean_squared_error: 0.7578 - val_loss: 0.5067 - val_root_mean_squared_er

2024/10/22 12:05:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run dazzling-fawn-971 at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1/runs/4595616025984e2e92c3eb7608000add.

2024/10/22 12:05:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1.



Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 25s 566ms/step - loss: 34.6086 - root_mean_squared_error: 5.8829
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.0385 - root_mean_squared_error: 5.8342   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 33.8716 - root_mean_squared_error: 5.8198 - val_loss: 32.0910 - val_root_mean_squared_error: 5.6649

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 334ms/step - loss: 33.9579 - root_mean_squared_error: 5.8273
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.7230 - root_mean_squared_error: 5.6320   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.4005 - root_mean_squared_error: 5.6033 - val_loss: 29.4218 - val_root_mean_squared_error: 5.4242

Epoch 3/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 6s 152ms/step - loss: 29.0626 - root_mean_squared_error: 5.3910
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.6543 - root_mean_squared_error: 5.3529  
44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.4730 - root_mean_squared_error: 5.3359
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4m

...

2024/10/22 12:06:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run adorable-gnat-556 at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1/runs/f9e3d724905e40a680539eb7920c3995.

2024/10/22 12:06:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1.



100%|██████████| 8/8 [02:00<00:00, 15.11s/trial, best loss: 0.7118229269981384]


2024/10/22 12:07:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run persistent-penguin-2 at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1/runs/e157b6ead8a9417597a0e6a38f8bc34d.


Best parameters: {'lr': 0.06562229465198674, 'momentum': 0.5667111424188149}
Best eval rmse: 0.7118229269981384


2024/10/22 12:07:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/loryana9/Collab_Dags.mlflow/#/experiments/1.


In [ ]:
# Load the model back for predictions as a generic Python Function model
mlflow.pyfunc.load_model( "models:/wine-quality/1")

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.tensorflow
  run_id: 78e4eb54ba8d4fb9a5dddd9ecdb0cb90